In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install py7zr
!pip install matplotlib

In [ ]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
dataset['train']

In [ ]:
dataset['train'][1]['article'][:300]


In [ ]:
dataset['train'][1]['highlights']

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="gpt2-medium")

In [ ]:
print(dataset['train'][1]['article'][:1000])


input_text = dataset['train'][1]['article'][:2000]
query = input_text + "\nTL;DR:\n"


pipe_out = pipe(query, max_length=512, clean_up_tokenization_spaces=True)


print(pipe_out[0]['generated_text'])

In [ ]:
len(input_text)

In [ ]:
pipe_out[0]['generated_text']

In [ ]:
pipe_out[0]['generated_text'][len(query):]

In [ ]:
summaries = {}
summaries['gpt2-medium-380M'] = pipe_out[0]['generated_text'][len(query):]

In [ ]:
pipe = pipeline('summarization',model='t5-base')

In [ ]:
pipe_out = pipe(input_text)

In [ ]:
summaries['t5-base-223M'] = pipe_out[0]['summary_text']

In [ ]:
summaries['bart-large-cnn-400M'] = pipe_out[0]['summary_text']

In [ ]:
pipe_out = pipe(input_text)

In [ ]:
summaries['pegasus-cnn-568M'] = pipe_out[0]['summary_text']

In [ ]:
for model in summaries:
    print(model.upper())
    print(summaries[model])
    print(" ")


In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_ckpt = 'facebook/bart-large-cnn'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

print(dataset)

In [ ]:
data1 = load_dataset('cnn_dailymail', '3.0.0')


print(data1['train'][0])

In [ ]:
import pandas as pd
dialogue_len = [len(x['article'].split()) for x in data1['train']]
summary_len = [len(x['highlights'].split()) for x in data1['train']]

data = pd.DataFrame({'Dialogue Length': dialogue_len, 'Summary Length': summary_len})
data.hist(figsize=(15, 5))

In [ ]:
def get_feature(batch):
    encoding = tokenizer(
        batch['article'],
        text_target=batch['highlights'],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    encoding['labels'] = encoding['input_ids'].copy()
    return encoding
